<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import goatools 
goatools.__version__

'v1.0.3+15.g6cb8790'

In [2]:
import pandas as pd

In [3]:
from goatools.obo_parser import GODag
from goatools.anno.gaf_reader import GafReader
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
#obo = GODag('../data/raw/ontology/go-basic.obo')

In [4]:
bg_list = pd.read_csv('../data/raw/ontology/sgd_costanzogenes',header=None)

In [27]:
obodag = GODag('../data/raw/ontology/go-basic.obo',optional_attrs={'relationship'})
objanno = GafReader('../data/raw/ontology/sgd.gaf',namespaces=set(['BP']))
#    ns2assoc = objanno.get_ns2assc()

ns2assoc_excl = objanno.get_ns2assc(  ev_exclude = {'HGI' , 'IGI'})

#bg = pd.read_csv('data/raw/ontology/', header=None)
sgd_info = pd.read_csv('../data/raw/ontology/SGD_features.tab', sep='\t',header=None)
#e_pcc.df['Systematic gene name'].tolist()#list(bg.iloc[:, 0])  # .to_list()

geneid2name = pd.Series(sgd_info.iloc[:,3].values,index=sgd_info.iloc[:,0]).to_dict()

geneids = [i for i,j in geneid2name.items() if j in bg_list[0].tolist()]
goeaobj = GOEnrichmentStudyNS(
    bg_list[0].tolist(),  # List of mouse protein-coding genes
    ns2assoc_excl,  # geneid/GO associations
    obodag,  # Ontologies
    propagate_counts=True,
    relationships=True,
    alpha=0.1,  # default significance cut-off
    methods=['fdr', 'bonferroni','fdr_bh','sm_bonferroni'], prt=None)

../data/raw/ontology/go-basic.obo: fmt(1.2) rel(2021-05-01) 47,284 GO Terms; optional_attrs(relationship)
HMS:0:00:01.159093  41,910 annotations READ: ../data/raw/ontology/sgd.gaf BP

Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a and negatively_regulates part_of positively_regulates regulates
 82%  4,782 of  5,813 population items found in association


46 GO IDs NOT FOUND IN ASSOCIATION: GO:0055114 GO:0034225 GO:0072363 GO:0018065 GO:1900473 GO:0048310 GO:0072321 GO:0009108 GO:1900062 GO:0061416 GO:1900526 GO:0035969 GO:1990872 GO:0000733 GO:0061393 GO:1903474 GO:1900525 GO:0035961 GO:0001324 GO:0035392 GO:0097236 GO:0061427 GO:1900524 GO:1902353 GO:1900008 GO:0001302 GO:1900462 GO:0090203 GO:0061404 GO:0035952 GO:0001320 GO:0071733 GO:0010978 GO:1900472 GO:0001319 GO:0090202 GO:0018192 GO:0035955 GO:0090579 GO:0001300 GO:0001308 GO:1900465 GO:0061392 GO:1902352 GO:1900466 GO:0051188


In [28]:
query = ['S000004798', 'S000005553', 'S000002248']

In [31]:
go_results_all = goeaobj.run_study(query)


Run BP Gene Ontology Analysis: current study set of 3 IDs ...100%      3 of      3 study items found in association
100%      3 of      3 study items found in population(5813)
Calculating 5,048 uncorrected p-values using fisher_scipy_stats
   5,048 GO terms are associated with  4,756 of  5,813 population items
      86 GO terms are associated with      3 of      3 study items


Generate p-value distribution for FDR based on resampling (this might take a while)
Sample 0 / 500: p-value 0.001013652500154252
Sample 10 / 500: p-value 0.0005160846378842216
Sample 20 / 500: p-value 0.0010319916829848402
Sample 30 / 500: p-value 0.0015477211658482314
Sample 40 / 500: p-value 0.0004378245914243236
Sample 50 / 500: p-value 0.0005160846378842216
Sample 60 / 500: p-value 0.0025786475671841736
Sample 70 / 500: p-value 0.0005160846378842216
Sample 80 / 500: p-value 0.0010319916829848402
Sample 90 / 500: p-value 0.003093844546727288
Sample 100 / 500: p-value 0.08711760010393112
Sample 110 / 500: p-value 0.02355654873210248
Sample 120 / 500: p-value 0.0025786475671841736
Sample 130 / 500: p-value 0.0010319916829848402
Sample 140 / 500: p-value 0.003093844546727288
Sample 150 / 500: p-value 0.0005160846378842216
Sample 160 / 500: p-value 0.0010319916829848402
Sample 170 / 500: p-value 0.0005160846378842216
Sample 180 / 500: p-value 0.0020632731170778414
Sample 190 / 500: p-va

  METHOD fdr:
       0 GO terms found significant (< 0.1=alpha) (  0 enriched +   0 purified): local fdr
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
  METHOD bonferroni:
       0 GO terms found significant (< 0.1=alpha) (  0 enriched +   0 purified): local bonferroni
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
  METHOD fdr_bh:
       0 GO terms found significant (< 0.1=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
  METHOD bonferroni:
       0 GO terms found significant (< 0.1=alpha) (  0 enriched +   0 purified): statsmodels bonferroni
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified

In [34]:
from fisher import pvalue
mat = [[2, 3], [103, 5813]]
p = pvalue(2,1,101,5709)
p.left_tail, p.right_tail, p.two_tail  # doctest: +NORMALIZE_WHITESPACE, +ELLIPSIS
#(0.04455473783507..., 0.994525206021..., 0.0802685520741...)

(0.999994595182878, 0.0009220852498519628, 0.0009220852498519628)

In [32]:
[r.__dict__ for r in go_results_all if r.GO=='GO:0006457']

[{'GO': 'GO:0006457',
  'name': 'protein folding',
  'NS': 'BP',
  'depth': 2,
  'method_flds': [NtMethodInfo(source='local', method='fdr', fieldname='fdr'),
   NtMethodInfo(source='local', method='bonferroni', fieldname='bonferroni'),
   NtMethodInfo(source='statsmodels', method='fdr_bh', fieldname='fdr_bh'),
   NtMethodInfo(source='statsmodels', method='bonferroni', fieldname='sm_bonferroni')],
  'kws': {'p_uncorrected': 0.0009220852498544861,
   'study_items': {'S000004798', 'S000005553'},
   'pop_items': {'S000000004',
    'S000000049',
    'S000000054',
    'S000000171',
    'S000000248',
    'S000000276',
    'S000000359',
    'S000000373',
    'S000000431',
    'S000000473',
    'S000000548',
    'S000000656',
    'S000000665',
    'S000000729',
    'S000000756',
    'S000000809',
    'S000000850',
    'S000000905',
    'S000001076',
    'S000001099',
    'S000001106',
    'S000001184',
    'S000001219',
    'S000001278',
    'S000001404',
    'S000001556',
    'S000001600',
   

In [26]:
[r.__dict__ for r in go_results_all if r.GO=='GO:0006457']

[{'GO': 'GO:0006457',
  'name': 'protein folding',
  'NS': 'BP',
  'depth': 2,
  'method_flds': [NtMethodInfo(source='local', method='fdr', fieldname='fdr'),
   NtMethodInfo(source='local', method='bonferroni', fieldname='bonferroni'),
   NtMethodInfo(source='statsmodels', method='fdr_bh', fieldname='fdr_bh'),
   NtMethodInfo(source='statsmodels', method='bonferroni', fieldname='sm_bonferroni')],
  'kws': {'p_uncorrected': 0.0005286611471875652,
   'study_items': {'S000004798', 'S000005553'},
   'pop_items': {'S000000004',
    'S000000049',
    'S000000054',
    'S000000276',
    'S000000359',
    'S000000373',
    'S000000431',
    'S000000473',
    'S000000548',
    'S000000656',
    'S000000729',
    'S000000756',
    'S000000809',
    'S000000850',
    'S000001076',
    'S000001099',
    'S000001184',
    'S000001219',
    'S000001278',
    'S000001404',
    'S000001600',
    'S000001678',
    'S000001937',
    'S000002302',
    'S000002371',
    'S000002562',
    'S000002575',
   

In [20]:
[r.__dict__ for r in go_results_all if r.GO=='GO:0019318']

[{'GO': 'GO:0019318',
  'name': 'hexose metabolic process',
  'NS': 'BP',
  'depth': 5,
  'method_flds': [NtMethodInfo(source='local', method='fdr', fieldname='fdr'),
   NtMethodInfo(source='local', method='bonferroni', fieldname='bonferroni'),
   NtMethodInfo(source='statsmodels', method='fdr_bh', fieldname='fdr_bh'),
   NtMethodInfo(source='statsmodels', method='bonferroni', fieldname='sm_bonferroni')],
  'kws': {'p_uncorrected': 0.0004629122390194999,
   'study_items': {'S000000222', 'S000003625'},
   'pop_items': {'S000000015',
    'S000000222',
    'S000000223',
    'S000000224',
    'S000000262',
    'S000000309',
    'S000000364',
    'S000000400',
    'S000000422',
    'S000000443',
    'S000000544',
    'S000000545',
    'S000000605',
    'S000000738',
    'S000000931',
    'S000000935',
    'S000001085',
    'S000001086',
    'S000001217',
    'S000001253',
    'S000001279',
    'S000001304',
    'S000001307',
    'S000001359',
    'S000001369',
    'S000001429',
    'S000001